In [1]:
import requests
import pprint as pp

try:
    response = requests.get("https://vigilant-surprise-production.up.railway.app/v1/models/stress_sleep_model")
    pp.pprint(response.json())
except requests.exceptions.RequestException as e:
    print("Error:", e)


{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1'}]}


In [2]:
import pandas as pd 

df = pd.read_csv('dataset/sayopillow.csv')
df

,snoring_rate,respiration_rate,body_temp,limb_movement,blood_oxygen,eye_movement,sleep_hours,heart_rate,stress_rate
0,93.800,25.680,91.840,16.600,89.840,99.60,1.840,74.20,medium_high
1,91.640,25.104,91.552,15.880,89.552,98.88,1.552,72.76,medium_high
2,60.000,20.000,96.000,10.000,95.000,85.00,7.000,60.00,medium_low
3,85.760,23.536,90.768,13.920,88.768,96.92,0.768,68.84,medium_high
4,48.120,17.248,97.872,6.496,96.248,72.48,8.248,53.12,low/normal
...,...,...,...,...,...,...,...,...,...
625,69.600,20.960,92.960,10.960,90.960,89.80,3.440,62.40,medium
626,48.440,17.376,98.064,6.752,96.376,73.76,8.376,53.44,low/normal
627,97.504,27.504,86.880,17.752,84.256,101.88,0.000,78.76,high/unhealthy
628,58.640,19.728,95.728,9.728,94.592,84.32,6.728,59.32,medium_low


In [3]:
low_normal_sample = df[df.stress_rate == 'low/normal'].sample(1)
med_sample = df[df.stress_rate == 'medium'].sample(1)
med_low_sample=df[df.stress_rate == 'medium_low'].sample(1)
med_high_sample = df[df.stress_rate == 'medium_high'].sample(1)
high_sample = df[df.stress_rate == 'high/unhealthy'].sample(1)

low_normal_sample = low_normal_sample.to_json(orient='records', lines=False)
med_sample = med_sample.to_json(orient='records', lines=False)
med_low_sample = med_low_sample.to_json(orient='records', lines=False)
med_high_sample = med_high_sample.to_json(orient='records', lines=False)
high_sample = high_sample.to_json(orient='records', lines=False)

print('low_normal', low_normal_sample)
print('med', med_sample)
print('med_low', med_low_sample)
print('med_high', med_high_sample)
print('high', high_sample)

low_normal [{"snoring_rate":46.28,"respiration_rate":16.512,"body_temp":96.768,"limb_movement":5.024,"blood_oxygen":95.512,"eye_movement":65.12,"sleep_hours":7.512,"heart_rate":51.28,"stress_rate":"low\/normal"}]
med [{"snoring_rate":65.12,"respiration_rate":20.512,"body_temp":92.512,"limb_movement":10.512,"blood_oxygen":90.512,"eye_movement":87.56,"sleep_hours":2.768,"heart_rate":61.28,"stress_rate":"medium"}]
med_low [{"snoring_rate":59.76,"respiration_rate":19.952,"body_temp":95.952,"limb_movement":9.952,"blood_oxygen":94.928,"eye_movement":84.88,"sleep_hours":6.952,"heart_rate":59.88,"stress_rate":"medium_low"}]
med_high [{"snoring_rate":82.4,"respiration_rate":22.64,"body_temp":90.32,"limb_movement":12.8,"blood_oxygen":88.32,"eye_movement":95.8,"sleep_hours":0.32,"heart_rate":66.6,"stress_rate":"medium_high"}]
high [{"snoring_rate":98.816,"respiration_rate":28.816,"body_temp":88.52,"limb_movement":18.408,"blood_oxygen":86.224,"eye_movement":103.52,"sleep_hours":0.0,"heart_rate":82

In [4]:
import tensorflow as tf
import json
import base64
import numpy as np 

features = ['snoring_rate', 'respiration_rate', 'body_temp', 'limb_movement',
       'blood_oxygen', 'eye_movement', 'sleep_hours', 'heart_rate']

# format input data to tf.example
def prepare_json(data):
    # feature dictionary 
    feature_spec = {
        feature: tf.train.Feature(float_list=tf.train.FloatList(value=[data[feature]]))
        for feature in features
    }

    # serialize to tf.example
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    # encode base64
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    # prepare JSON
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })
    
def make_predictions(data):
    
    json_data = prepare_json(data)
    endpoint = 'http://localhost:8501/v1/models/stress_sleep_model:predict'
    response = requests.post(endpoint, data=json_data, headers={"Content-Type": "application/json"})

    prediction = response.json()["predictions"][0]
    
    max_index = np.argmax(prediction)
    
    index = {
        0 : 'high', 
        1 : 'low - normal',
        2 : 'medium',
        3 : 'medium high',
        4 : 'medium low'
    }
    
    stress_level = index[max_index]
    
    return max_index, stress_level

low_normal = {
    "snoring_rate":45.6,
    "respiration_rate":16.24,
    "body_temp":96.36,
    "limb_movement":4.48,
    "blood_oxygen":95.24,
    "eye_movement":62.4,
    "sleep_hours":7.24,
    "heart_rate":50.6,
    # "stress_rate":"low\/normal"
    }
med =  {
    "snoring_rate":71.04,
    "respiration_rate":21.104,
    "body_temp":93.104,
    "limb_movement":11.104,
    "blood_oxygen":91.104,
    "eye_movement":90.52,
    "sleep_hours":3.656,
    "heart_rate":62.76,
    # "stress_rate":"medium"
    }
med_low = {
    "snoring_rate":58.4,
    "respiration_rate":19.68,
    "body_temp":95.68,
    "limb_movement":9.68,
    "blood_oxygen":94.52,
    "eye_movement":84.2,
    "sleep_hours":6.68,
    "heart_rate":59.2,
    # "stress_rate":"medium_low"
    }
med_high = {
    "snoring_rate":94.88,
    "respiration_rate":25.968,
    "body_temp":91.984,
    "limb_movement":16.96,
    "blood_oxygen":89.984,
    "eye_movement":99.96,
    "sleep_hours":1.984,
    "heart_rate":74.92,
    # "stress_rate":"medium_high"
    }
high = {
    "snoring_rate":99.2,
    "respiration_rate":29.2,
    "body_temp":89.0,
    "limb_movement":18.6,
    "blood_oxygen":86.8,
    "eye_movement":104.0,
    "sleep_hours":0.0,
    "heart_rate":83.0,
    # "stress_rate":"high\/unhealthy"
    }

2024-11-05 00:25:50.864615: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-05 00:25:51.167845: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-05 00:25:51.167911: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-05 00:25:51.231233: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 00:25:51.506765: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-05 00:25:51.510719: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
# make predictions here
pred = make_predictions(med)
print(pred)

(2, 'medium')


In [6]:
import random
import time

# perform 100 requests to populate request
data_categories = [low_normal, med, med_low, med_high, high]

for _ in range(100):
    selected_data = random.choice(data_categories)
    max_index, stress_level = make_predictions(selected_data)    
    print(f"Stress Level: {stress_level}")
    time.sleep(0.2)

Stress Level: medium
Stress Level: high
Stress Level: low - normal
Stress Level: medium high
Stress Level: high
Stress Level: medium high
Stress Level: high
Stress Level: medium
Stress Level: low - normal
Stress Level: medium
Stress Level: high
Stress Level: medium low
Stress Level: medium high
Stress Level: high
Stress Level: low - normal
Stress Level: medium low
Stress Level: high
Stress Level: medium high
Stress Level: medium low
Stress Level: medium
Stress Level: medium high
Stress Level: medium low
Stress Level: medium high
Stress Level: medium
Stress Level: medium low
Stress Level: low - normal
Stress Level: medium low
Stress Level: low - normal
Stress Level: high
Stress Level: low - normal
Stress Level: medium
Stress Level: medium high
Stress Level: low - normal
Stress Level: medium
Stress Level: medium high
Stress Level: medium high
Stress Level: medium low
Stress Level: medium low
Stress Level: low - normal
Stress Level: medium high
Stress Level: medium
Stress Level: medium lo